Notebook to calculate grid adjacency and do p-class clustering for Rosi's MPAS aquaplanet grid.

Also writes out a single time step of p-class values.

James Ruppert  
9/1/2025

### Main settings

In [1]:
import numpy as np
import pickle
import matplotlib.pyplot as plt
from matplotlib import rc, colors
import cartopy.crs as ccrs

In [2]:
do_write = True
do_write = False

pclass_names = ['DC', 'CG', 'SC', 'ST', 'AN', 'DSA']
nclass = len(pclass_names)
exp_names = ["CTL", "HOMO_RAD", "CLIM_RAD"]
expName=exp_names[0]

pickle_dir = f"/glade/derecho/scratch/ruppert/tc-crfrad/pickle_out"
# pickle_dir = f"../../../pickle_out/aquaplanet"

### Read/Write variable from MPAS output

In [3]:
# Read grid data
do_write_grid = False
grid_pickle_file = f"{pickle_dir}/grid_data.pickle"
if do_write_grid:
    import xarray as xr
    grid_path = "/glade/work/rberrios/MPAS/aqua_sstmax10N_ASD/plus4K/TC_3km/x5.tropical_3km_10N.init.nc"
    grid = xr.open_dataset(grid_path)
    areaCell = grid.areaCell
    latCell = np.degrees(grid.latCell)
    lonCell = np.degrees(grid.lonCell)
    with open(grid_pickle_file, 'wb') as f:
        pickle.dump([areaCell, latCell, lonCell], f)
else:
    with open(grid_pickle_file, 'rb') as f:
        areaCell, latCell, lonCell = pickle.load(f)

In [4]:
# One-time jobs

if do_write:

    import xarray as xr
    import dask.array as da
    import dask
    from precip_class_mpas import *

    model_path = "/glade/campaign/mmm/dpm/rberrios/glade_scratch/MPAS_APE/aqua_sstmax10N_ASD/"

    # Get list of desired file times
    file_times_arr = np.arange('2000-05-01T06:00:00', '2000-05-11T06:00:00', 6, dtype='datetime64[h]')
    file_times = [file_times_arr[i].astype('datetime64[D]').astype(str)+'_'+str(file_times_arr[i]).split('T')[1].split(':')[0]+'.00.00' for i in range(len(file_times_arr))]

In [5]:
# Possible looping over time

if do_write:

    for it in range(0,len(file_times),5):
        
        time = file_times[it]
        # if it == 20:
        #     continue

        # Choose experiment and read in model data
        data_path = f"{model_path}{expName}/TC_3km/"
        wp_files = data_path+'waterPaths.'+time+'.nc'
        nCells_chunk_size = 100000
        ds = xr.open_mfdataset(wp_files,
                    parallel=True, 
                    chunks={"Time": -1, "nCells": nCells_chunk_size})

        q_int_dask = da.stack([
            ds.lwp.data,
            ds.iwp.data,
            ds.rwp.data,
            ds.gwp.data
        ], axis=0) # Stack along a new 0th dimension for the different water paths

        c_type_dask = precip_class_mpas(q_int_dask)
        c_type = dask.compute(c_type_dask)[0][0] # dask.compute returns a tuple of results

        # Write to pickle
        pickle_file_out = f"{pickle_dir}/pclass_{expName}_{time}.pickle"
        with open(pickle_file_out, 'wb') as f:
            # pickle.dump(PE_thisExp, f)
            # pickle.dump([latCell, lonCell, c_type], f)
            pickle.dump(c_type, f)

In [6]:
# Read list of files from a directory
# import glob
# file_list = glob.glob(f"{pickle_dir}/pclass_{expName}*pickle")

# Get list of desired file times
file_times_arr = np.arange('2000-05-01T06:00:00', '2000-05-11T06:00:00', 6, dtype='datetime64[h]')
file_times = [file_times_arr[i].astype('datetime64[D]').astype(str)+'_'+str(file_times_arr[i]).split('T')[1].split(':')[0]+'.00.00' for i in range(len(file_times_arr))]
file_times_read = [file_times[i] for i in range(0,len(file_times),5)]

# time = file_times[20]

# Write to pickle
# Read from pickle
c_type = []
for time in file_times_read:
    pickle_file_read = f"{pickle_dir}/pclass_{expName}_{time}.pickle"
    with open(pickle_file_read, 'rb') as f:
        c_type.append(pickle.load(f))
c_type = np.array(c_type)

### Plot P-Class

In [13]:
ll_bounds = (0,60,0,30)
filter = np.where((lonCell >= ll_bounds[0]) &
                  (lonCell <= ll_bounds[1]) &
                  (latCell >= ll_bounds[2]) &
                  (latCell <= ll_bounds[3]))[0]

font = {'family' : 'sans-serif',
        'weight' : 'normal',
        'size'   : 12}
rc('font', **font)

In [11]:
bounds = np.array([-0.5, .5, 1.5, 2.5, 3.5, 4.5, 5.5])
norm = colors.BoundaryNorm(boundaries=bounds, ncolors=6)

nlevs = len(bounds) - 1
cmap = plt.get_cmap('Accent_r', nlevs)

for it, itime in enumerate(file_times_read):

    fig = plt.figure(figsize=(12,10))
    fig.set_facecolor('white')
    ax = fig.add_subplot(111, projection=ccrs.Mollweide())
    ax.gridlines()#draw_labels=True, x_inline=False, y_inline=False)#, dms=True)

    # sc = ax.scatter(lonCell[filter], latCell[filter], c=c_type[filter], cmap=cmap, norm=norm, s=1)
    sc = ax.scatter(lonCell, latCell, c=c_type[it], cmap=cmap, norm=norm, s=1, transform=ccrs.PlateCarree())
    # sc = ax.contourf(lonCell, latCell, c=c_type, cmap=cmap, norm=norm, s=1)
    # sc = ax.tricontourf(lonCell, latCell, c_type, levels=15, cmap='viridis')

    cbar=plt.colorbar(sc, fraction=0.03, pad=0.02, ticks=(0,1,2,3,4,5), shrink=0.65)
    cbar.set_ticklabels(['Nonraining','Deep', 'Congestus', 'Shallow', 'Stratiform', 'Anvil'])
    cbar.ax.tick_params(labelsize=14)
    ax.set_title(f"Cloud Classification ({expName}, {itime})", fontsize=16)
    # plt.ylim((-5,25))

    plt.tight_layout()
    plt.show()
    plt.savefig(f"pclass_map_{expName}_{itime}.png", dpi=200)